### **1 - Importing Libraries**

In [ ]:
import pandas as pd
import json
from unsloth import FastLanguageModel
from trl import SFTTrainer
from unsloth import is_bfloat16_supported
from huggingface_hub import login
from transformers import TrainingArguments 
import wandb
from datasets import Dataset, DatasetDict

### **2 - Loading Configuration**

In [2]:
with open('config.json', 'r') as file:
    config = json.load(file)

# general configuration
HGF = config['general']['HGF']
WNB = config['general']['WNB']

# outputs
output_model_online = config['outputs']['output_model_online_Desc']
output_model_local = config['outputs']['output_model_local_Desc']

# model
base_model = config['model']['base_model']
max_seq_length = config['model']['max_seq_length']
load_in_4bit = config['model']['load_in_4bit']

# lora_config
r = config['lora_config']['r']

# fine_tuning
dataset_num_proc = config['fine_tuning']['dataset_num_proc']
per_device_train_batch_size = config['fine_tuning']['per_device_train_batch_size']
gradient_accumulation_steps =  config['fine_tuning']['gradient_accumulation_steps']
epochs =  config['fine_tuning']['epochs']['descriptor']
max_steps = config['fine_tuning']['max_steps']
warmup_steps = config['fine_tuning']['warmup_steps']
learning_rate = config['fine_tuning']['learning_rate']
optim = config['fine_tuning']['optim']
weight_decay = config['fine_tuning']['weight_decay']
lr_scheduler_type = config['fine_tuning']['lr_scheduler_type']
output_dir = config['fine_tuning']['output_dir']

### **3 - Reading Data**

In [3]:
train_df = pd.read_csv("Data/MIND-Preprocessed/train.csv", index_col=0)
valid_df = pd.read_csv("Data/MIND-Preprocessed/valid.csv", index_col=0)

### **4 - Authentication & Experiment Tracking**

In [ ]:
login(HGF)
wandb.login(key=WNB)
run = wandb.init(
    project = "Description-Generator-MIND",
    name = '5-epochs'
    job_type="training",
    anonymous="allow"
)

### **5- Loading DeepSeek R1 : Model & Tokenizer**

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = base_model,
    max_seq_length = max_seq_length,
    load_in_4bit = load_in_4bit,
    token = HGF
)

### **6 - Setting Up Dataset**

In [ ]:
temp_1 = Dataset.from_pandas(train_df)
temp_2 = Dataset.from_pandas(valid_df)

dataset = DatasetDict({
    "train": temp_1,
    "validation": temp_2,
    
})

In [ ]:
prompt_style = """ Below is an instruction that describes a task, paired with an input that provied further context.
Write a response that appropiately completes the request.

### Instruction:
You are an interests analyzer. Based on the following user history, analyze their reading habits and generate a description of what kind of news articles they might be interested in reading next. 

### History:
{}

### Response:
Description : \n
{}

"""

In [ ]:
EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    inputs = examples["history"]
    outputs = examples["Description"]
    prompts = []
    for input, output in zip(inputs, outputs):
        prompt = prompt_style.format(input, output) + EOS_TOKEN
        prompts.append(prompt)
    return {
        "prompt": prompts,
    }

In [ ]:
dataset = dataset.remove_columns(['candidate',"label"])

In [ ]:
dataset_finetune = dataset.map(formatting_prompts_func, batched = True)
dataset_finetune

### **7 - Setting up the model using LORA**

In [19]:
# Apply LoRA (Low-Rank Adaptaion) fine-tuning to the model
model_lora = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules=[ # listing the tarnsfomers layer where lora will be applied 
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,  
    bias="none",  
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=3407,
    use_rslora=False,  
    loftq_config=None,
)

Unsloth 2025.3.14 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


### **8 - Fine-Tuning: Setup and Training**

In [ ]:
trainer = SFTTrainer(
    model=model_lora,
    tokenizer=tokenizer,
    train_dataset=dataset_finetune['train'],
    eval_dataset=dataset_finetune['validation'],
    dataset_text_field="prompt",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=per_device_train_batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        num_train_epochs=epochs,
        max_steps=max_steps,
        warmup_steps=warmup_steps,
        learning_rate=learning_rate,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=20,
        optim=optim,
        weight_decay= weight_decay,
        lr_scheduler_type=lr_scheduler_type,
        seed=777,
        output_dir=output_dir,
    ),
)

In [ ]:
trainer.train()

### **9 - Saving Online**

In [ ]:
model_lora.push_to_hub(output_model_online) 
tokenizer.push_to_hub(output_model_online)

### **10 - Saving Locally**

In [ ]:
model.save_pretrained(output_model_local) 
tokenizer.save_pretrained(output_model_local)

In [ ]:
wandb.finish()

### **11 - Generating Descriptions for Test Data**

In [6]:
test_df = pd.read_csv("Data/test.csv", index_col=0)

In [ ]:
def generate_Descriptions(df,model):

    history = []
    desc = []
    candidates = []
    labels = []

    for (i, row) in df.iterrows():
           
            prompt = row['history']
            inputs = tokenizer([prompt_style.format(prompt, "")], return_tensors="pt").to("cuda")
        
            outputs = model.generate(
                input_ids=inputs.input_ids,
                attention_mask=inputs.attention_mask,
                max_new_tokens= 900
            )
            response = tokenizer.batch_decode(outputs)
            result = response[0].split("### Response:")[1].split("\nDescription : \n\n\n\n")[1].replace("<｜end▁of▁sentence｜>","")
            
            desc.append(result)
            candidates.append(row['candidate'])
            labels.append(row['label'])
            history.append(prompt)
            
    return history, desc, candidates, labels

In [ ]:
h,d,c,l = generate_Descriptions(test_df, model_lora)

In [ ]:
df = pd.DataFrame({
    "history":h,
    "Descriptions":d,
    "Candidates":c,
    "Labels":l
})

In [33]:
df['Descriptions'] = df['Descriptions'].str.replace('\n\n', '', regex=False)

In [ ]:
df.to_csv('Data/MIND-Preprocessed/test.csv', index=False)